In [2]:
import cv2
import pandas as pd
from pathlib import Path
import json

In [3]:
DATA_DIR = Path('data/')

In [4]:
!git clone -b u7 --single-branch https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 282, done.
remote: Total 282 (delta 0), reused 0 (delta 0), pack-reused 282
Receiving objects: 100% (282/282), 15.26 MiB | 233.00 KiB/s, done.
Resolving deltas: 100% (103/103), done.


In [ ]:
#!pip install -r yolov7/seg/requirements.txt

In [ ]:
# !mkdir yolo_wheel
# !pip wheel -r yolov7/seg/requirements.txt --wheel-dir=yolo_wheel yolov7

In [ ]:
# from yolov7.seg.segment import train

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

# I have saved my API token with "WANDB" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = user_secrets.get_secret("WANDB") 

wandb.login(key=wandb_api)

In [ ]:
# Create a yaml file as expected by YOLOv7 (and others)
yaml_text = """
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for high-augmentation COCO training from scratch
# python train.py --batch 32 --cfg yolov5m6.yaml --weights '' --data coco.yaml --img 1280 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.0005  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.3  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.9  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.1  # image mixup (probability)
copy_paste: 0.1  # segment copy-paste (probability)
"""
with open('/kaggle/working/hyp.yaml', 'w') as text_file:
    text_file.write(yaml_text)

#### Training

In [ ]:
train.run(data='/kaggle/input/hubmap-hhv-coco/hubmap-coco.yaml',
          imgsz=512, 
          batch=16,
          weights='yolov7-seg.pt',
          cfg='yolov7/seg/models/segment/yolov7-seg.yaml',
          epochs=25,
          name='yolov7-fine-tune',
          project='yolov7-fine-tune',
          hyp='/kaggle/working/hyp.yaml',
          optimizer='Adam'
          )